In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
data = pd.read_csv("../input/nlp-getting-started/train.csv")

In [ ]:
data.head()

In [ ]:
100 * data.isna().sum() / data.shape[0]

In [ ]:
data["location"].value_counts()

In [ ]:
data["keyword"].value_counts()

In [ ]:
data.drop(["keyword", "location", "id"], axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
style.use("fivethirtyeight")

In [ ]:
sns.countplot(x='target', data=data)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(data["text"], data["target"], test_size=0.15)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(oov_token="oov", num_words=18000)

In [ ]:
tokenizer.fit_on_texts(data["text"])

In [ ]:
len(tokenizer.word_index)

In [ ]:
xtrain=tokenizer.texts_to_sequences(xtrain)
xtrain = pad_sequences(sequences=xtrain, maxlen=30)

xtest=tokenizer.texts_to_sequences(xtest)
xtest = pad_sequences(sequences=xtest, maxlen=30)

In [ ]:
from keras import layers
from tensorflow import keras
from tensorflow.compat.v1.keras.layers import CuDNNGRU as gru

In [ ]:
def build_model(embdim=512, emb_dropout=0.2, dropout_rate=0.25, reg=0.0001):
    i = layers.Input(shape=(30,))
    x = layers.Embedding(15000,embdim)(i)
    x = layers.Dropout(emb_dropout)(x)
    x = layers.Bidirectional(gru(128,
                                 return_sequences=True,
                                 kernel_regularizer=tf.keras.regularizers.l1(reg),
                                 activity_regularizer=tf.keras.regularizers.l1(reg)))(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Bidirectional(gru(64,
                                 return_sequences=True,
                                 kernel_regularizer=tf.keras.regularizers.l1(reg),
                                 activity_regularizer=tf.keras.regularizers.l1(reg)))(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Bidirectional(gru(64,
                                 kernel_regularizer=tf.keras.regularizers.l1(reg),
                                 activity_regularizer=tf.keras.regularizers.l1(reg)))(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inputs=i, outputs=x)

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
import tensorflow_addons as tfa

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=5, batch_size=32)